# Neste notebook vamos definir as funçoes de filtragem e visualizaçao dos dados a partir das classes definidas na bilbioteca GeoPandas e Matplotlib

[Mapping and Plotting Tools](https://geopandas.org/docs/user_guide/mapping.html)

[Matplotlib](https://matplotlib.org/stable/tutorials/introductory/usage.html#sphx-glr-tutorials-introductory-usage-py)

- Com as funçoes definidas e testadas neste notebook, criaremos um arquivo .py no diretorio .soureces/ para armazena-las, possibilitando assim, que possamos chama-las com a funçao import.

In [1]:
import geopandas as gpd
#import fiona

In [2]:
# Origem do banco de dados
gdb = '/home/ggrl/geodatabase/'

# Parametro para plot interativo
%matplotlib widget

## Importando um vetor do banco de dados criado no notebook passado.
- Escolha de camada e mapa
    - Caso nao tenha o mapa selecionado retorne lista de mapas disponiveis para aquela escala.
    - Caso nao escolha um mapa retorne todos os vetores daquela camada.

In [3]:
# IMPORTADOR DE LITOLOGIAS POR ESCALA --------------------------------------------------------------------------#
def importar_geometrias(camada, mapa=False):
    '''
    Recebe:
        camada      : camada vetorial a ser lida do geopackage.
        mapa        : nome do mapa presente na camada vetorial;

    Retorna:
        Objeto GeoDataFrame.
        
    Se houver seleçao de mapa retornara apenas as geometrias que possuem o nome escolhido na coluna ['MAPA']
    Se Retornar camada vazia recebera a lista das camadas veotoriais diposniveis
    Se mapa == False: retorna todos os objetos presente nesta camada vetorial
    '''
    lito =  gpd.read_file(gdb+'geodatabase.gpkg',
                        driver= 'GPKG',
                        layer= camada)
                        
    #print(f"# -- Lista de camadas vetoriais disponiveis: {list(fiona.listlayers(gdb+'geodatabase.gpkg'))}")  

    if mapa:
        folha = lito[lito.MAPA == 'Carta geológica da folha '+mapa]
        if folha.empty:
            print(f"O mapa escolhido nao est'a presente na coluna MAPA da camada veotiral.")
            print(f"# -- Lista de mapas: {list(lito.MAPA.unique())}")

        else:    
            return(folha)
    else:
        return(lito)

##  Utilizando as funçoes definidas acima para importar os dados

In [4]:
# Abrindo os vetores descritivos dos projetos aerogeofisicos
proj_geof = importar_geometrias ('proj_aerogeof')

In [5]:
litologia_100k = importar_geometrias('litologia_100k')

In [6]:
print(proj_geof.shape)
print(litologia_100k.shape)

(296, 17)
(59268, 27)


In [31]:
proj_geof['geometry'].values

<GeometryArray>
 ...
Length: 296, dtype: geometry

In [29]:
fig, axs = plt.subplots()
ax.plot(proj_geof['geometry'],label='proj')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: The dataframe and given column have different number of rows.

## Calculando area de cobertura dos projetos aerogeofisicos

In [ ]:
a = litologia_100k.bounds

In [ ]:
a

In [ ]:
a['minx']

In [ ]:
a.plot()

In [ ]:
#b = a.unary_union

In [ ]:
proj_250m = proj_geof[proj_geof['ESPACAMENT'] == '250']

In [ ]:
print(proj_250m['geometry'][16])

In [ ]:
litologia_100k

In [ ]:
proj_geof_500m = proj_geof[proj_geof['ESPACAMENT'] == '500']

In [ ]:
print(len(proj_geof_500m))
proj_geof_500m['DISPONIBIL'].isnull().sum()

In [ ]:
proj_geof_500m.columns

In [ ]:
dic_meta_litologico = {'lista_colunas' : '',}
dic_meta_geofisico = {'lista_colunas' : '',}

In [ ]:
print(dic_meta_litologico.keys())

In [ ]:
x = list(proj_geof_500m.columns)
dic_meta_litologico.update({'lista_colunas':x})

In [ ]:
print(dic_meta_litologico['lista_colunas'])

In [ ]:
dic_meta_litologico['lista_colunas']

In [ ]:
#proj_geof_500m['SHAPE_AREA'].sum() / 

In [ ]:
proj_geof.columns

In [ ]:
#proj_geof.geometry.area.sum()

In [ ]:
print(type(proj_geof['ESPACAMENT'][0]))
print(list(proj_geof.ESPACAMENT.unique()))
print(list(proj_geof.ESPACAMENT.unique()))

In [ ]:
lista_espacament = list(proj_geof.ESPACAMENT.unique())

In [ ]:
for i in proj_geof['ESPACAMENT']:
    if i str.contains('.' or ','):
        print(i)
    else:
        proj_geof[proj_geof['ESPACAMENT'].i]

    break


In [ ]:
print(f"Foram identificados {len(proj_geof[proj_geof['ESPACAMENT'] == '500'])} projetos aerogeofisicos")
print(f"Lista de Titulos: {proj_geof[proj_geof['ESPACAMENT'] == '500']['TITULO']}")

In [ ]:
proj_geof_500m = proj_geof[proj_geof['ESPACAMENT'] == '500']

In [ ]:
# Abrindo os vetores litológicos utilizando a função definida acima
litologia_100k = importar_geometrias('litologia_100k')
litologia_250k = importar_geometrias('litologia_250k')

In [ ]:
plot_brazil(proj_geof_500m.boundary)
plt.suptitle('500 metros')
plt.annotate('ID_PROJETO',)
plt.axis('scaled')

In [ ]:
#int(proj_geof['ESPACAMENT'])

In [ ]:
proj_geof_500m = proj_geof[proj_geof.ESPACAMENT == '500']

In [ ]:
#ax = litologia_100k.plot('SIGLA')
#proj_geof_500m.boundary.plot(ax=ax)

plot_base(proj_geof_500m.boundary,camada='litologia_100k')  # esta funçao e lenta pois importa toda a camada vetorial novamente e
                                                            # nao deve ser usada caso a litologia ja tenhasido definida

## Listando atributos geologicos das geometrias importadas.

In [ ]:
# Abrindo os vetores litológicos utilizando a função definida acima
litologia_100k = importar_geologico('litologia_100k')
litologia_250k = importar_geologico('litologia_250k')

In [ ]:
# Abrindo os vetores litológicos de um unico mapa utilizando a função definida acima
varginha_100k = importar_geologico('litologia_100k','Varginha - UFRJ')

In [ ]:
# atributos descritivos de cada uma das camadas litológicas (geometrias)
print("Lista de atributos descritivos classes litologicas: ")
print(list(varginha_100k.columns))

In [ ]:
# Plotando os vetores descritos pelos geólogos da cprm
varginha_100k.plot('SIGLA')

In [ ]:
print(varginha_100k.shape)
print(f"{varginha_100k.shape[0]} geometrias litologicas descritas em {varginha_100k.shape[1]} atributos")

In [ ]:
# Observando o formato da descrição da coluna litotipos
varginha_100k_litotipos = varginha_100k.LITOTIPOS.unique()
print(f"{len(list(varginha_100k_litotipos))} litotipos reconhecidos")
list(varginha_100k_litotipos)

In [ ]:
# com estas funçoes utilizadas asssimas, podemos definir uma funçao que descreve o nosso dado vetorial
def describe_geologico(gdf):
    lista_colunas = list(gdf.columns)
    lista_shape = gdf.shape
    lista_litotipos = list(gdf.LITOTIPOS.unique())
    lista_legenda = list(gdf.LEGENDA.unique())

    dic_litologico = {'lista_colunas': lista_colunas,
                      'lista_shape': lista_shape,
                      'lista_litotipos': lista_litotipos,
                      'lista_legenda': lista_legenda}
    return dic_litologico   

In [ ]:
dic_litoligico = describe_geologico(varginha_100k)

In [ ]:
print(f"Numero de geometrias descritas na carta geologica: {len(dic_litoligico['lista_litotipos'])}")
print("Lista de valores unicos da coluna Litotipo: ")
list(dic_litoligico['lista_litotipos'])

In [ ]:
print(f"Numero de geometrias descritas na carta geologica: {len(dic_litoligico['lista_legenda'])}")
print("Lista de valores unicos da coluna Legenda: ")
list(dic_litoligico['lista_legenda'])

## Reconhecendo as litologia mapeadas em suas escalas
- Faremos um filtro na coluna LEGENDA e LITOTIOPOS para selecionar as classes de interesse e, com estas geometrias, selecionaremos os mapas que as contém.
- Assim podemos identificar quais cartas geológicas e suas escalas podemos trabalhar.

In [ ]:
def filtro(gdf,mineral):
    filtrado = gdf[gdf['LITOTIPOS'].str.contains(mineral)]
    if filtrado.empty:
        print(f"{list(gdf['LITOTIPOS'].unique())}")
    else:
        return filtrado

In [ ]:
litologia_grafitosa_100k = filtro(litologia_100k,'graf')

In [ ]:
litotipos_vazio = litologia_100k[litologia_100k.LITOTIPOS.isna()]
litotipos_vazio.LITOTIPOS

In [ ]:
#Foi necessário inserir a str 'Vazio' em celas que não continham valores para que a função de filtro funcionasse corretamente na coluna LEGENDA.

#litologia_1kk.fillna('Vazio',inplace=True)
#litologia_250k.fillna('Vazio',inplace=True)
litologia_100k.fillna('Vazio',inplace=True)
#litologia_50k.fillna('Vazio',inplace=True)

In [ ]:
Grafita_l1kk = filtro(litologia_1kk,'Graf')
print(list(Grafita_l1kk['LITOTIPOS'].unique()))

In [ ]:
Grafita_l1kk.plot('SIGLA')

In [ ]:
base = brazil.boundary.plot(color='black',figsize=(12,12))

leg_grafitosa_l1kk.plot(ax=base,facecolor='grey')

lit_grafitoso_1kk.plot(ax=base,facecolor='black')

In [ ]:
list_gr_100k=mapas_lito_gr_100k
indice=0
for i in mapas_lito_gr_100k:
    print(i)
    list_gr_100k[indice] = l_100k[l_100k['MAPA'].str.contains("i")]
    indice+=1


In [ ]:
len(list_gr_100k)


### Desafio desnecessário para provar meus conhecimentos com for loops:

In [ ]:
lista_mapa = list(litologia_1kk.MAPA.unique())
print(len(lista_mapa))
lista_nome=[]

for mapa in lista_mapa:
    trash=0
    Mapa=[]
    for letra in mapa:
        if trash<25:
            trash+=1
        else:
            Mapa+=letra
    lista_nome += [''.join(Mapa)]

In [ ]:
lista_nome

In [ ]:
folha=lista_nome
indice=0
for i in lista_nome:
    print(i)
    folha[indice] = litologia_1kk[l_1kk['MAPA'].str.contains(i)].dissolve('MAPA')
    indice+=1

In [ ]:
folha[31].plot()

In [ ]:
lista_mapa = list(litolo_100k.MAPA.unique())
lista_mapa

In [ ]:
lista_nome=[]
for mapa in lista_mapa:
    trash=0
    Mapa=[]
    for letra in mapa:
        if trash<25:
            trash+=1
        else:
            Mapa+=letra
    lista_nome += [''.join(Mapa)]

In [ ]:
len(lista_nome)

In [ ]:
folha=lista_nome
indice=0
for i in lista_nome:
    print(i)
    folha[indice] = l_100k[l_100k['MAPA'].str.contains(i)].dissolve('MAPA')
    indice+=1

In [ ]:
len(folha)